# 1- Modelo TF-IDF + Regresión Logística para análisis de sentimiento en español

El modelo **TF-IDF + Regresión Logística** es un enfoque clásico y ampliamente utilizado en Procesamiento de Lenguaje Natural (NLP) para tareas de clasificación de texto, como el análisis de sentimiento.

Consiste en dos componentes principales:

**TF-IDF**: transforma el texto en una representación numérica basada en la importancia de las palabras.

**Regresión Logística**: utiliza esa representación numérica para aprender un modelo de clasificación que predice la clase de sentimiento (por ejemplo: positivo, negativo o neutro).

Este enfoque es especialmente efectivo cuando se trabaja con textos en español, datasets medianos o grandes y cuando se requiere un modelo interpretable y eficiente.

## 1.1- TF-IDF  (Term Frequency – Inverse Document Frequency)

**TF-IDF (Term Frequency – Inverse Document Frequency)**: s una técnica de vectorización de texto que cuantifica la importancia de cada palabra dentro de un documento, considerando también su relevancia en todo el corpus.

**TF – Term Frequency**: Mide qué tan frecuente es una palabra dentro de un documento.

Ejemplo: Si la palabra "*servicio*" aparece 3 veces en un comentario de 10 palabras, entonces TF(*servicio*) = 3 / 10.



**IDF – Inverse Document Frequency**: Mide qué tan exclusiva es una palabra en todo el corpus.
- Palabras comunes como "el", "y", "es" aparecen en casi todos los textos → bajo peso.
- Palabras específicas como "deficiente", "excelente" → alto peso.

Fórmula conceptual:

IDF(palabra) = log(total_docs / docs_que_contienen_la_palabra)

**TF-IDF = TF x IDF**

Resultado:

- Palabras frecuentes en un texto, pero raras en el conjunto total tienen mayor peso.

Esto es ideal para sentimiento, porque palabras emocionales son más informativas.

**¿Qué produce TF-IDF?**

Convierte cada comentario en un vector numérico de alta dimensión:

- Cada columna = una palabra del vocabulario

- Cada fila = un comentario

- Cada valor = peso TF-IDF

Ejemplo:

| comentario | excelente | malo | servicio | rápido |
| ---------- | --------- | ---- | -------- | ------ |
| c1         | 0.8       | 0.0  | 0.2      | 0.5    |
| c2         | 0.0       | 0.9  | 0.3      | 0.0    |

## 1.2- Regresión Logistica

La Regresión Logística es un modelo de aprendizaje supervisado utilizado para problemas de clasificación.

Aunque su nombre contiene “regresión”, su propósito principal es estimar probabilidades de pertenencia a una clase.

### 1.2.1- Funcionamiento conceptual

La Regresión Logística:

1. Recibe un vector de características (en este caso, el vector TF-IDF).
2. Calcula una combinación lineal de esas características.
3. Aplica una función logística (sigmoide o softmax).
4. Devuelve una probabilidad por clase.

La clase con mayor probabilidad es la predicción final.

### 1.2.2- Multiclase en análisis de sentimiento

Para análisis de sentimiento con tres clases (positivo, negativo, neutro), la Regresión Logística se extiende mediante: **One-vs-Rest** (OvR) o **Softmax multinomial**.

## 1.3- ¿Qué hace el modelo TF-IDF + Regresión Logística?

El modelo completo realiza el siguiente flujo:

1. Entrada: comentario en texto libre (en español).
2. Vectorización: el texto se transforma en un vector TF-IDF.
3. Clasificación: la Regresión Logística evalúa el vector.
4. Salida:
    - Etiqueta de sentimiento (positivo, negativo o neutro).
    - Score o probabilidad asociada a la predicción.

En esencia, el modelo aprende qué palabras y combinaciones de palabras están asociadas a cada sentimiento y utiliza esa información para clasificar nuevos textos.

## 1.3- ¿Por qué utilizar la combinación TF-IDF + Regresión Logística?

- Rápida de entrenar
- Muy eficiente para texto
- Buen rendimiento con datasets medianos
- Fácil de interpretar
- Ideal para APIs y producción (como FastAPI)

# 2- Modelo TF-IDF + Regresión Logística

In [1]:
# Manejo del DataFrame
import pandas as pd

import numpy as np

import matplotlib.pyplot as plt

# Convierte texto en vectores numéricos TF-IDF.
#Incluye tokenización, conteo y ponderación automática.
from sklearn.feature_extraction.text import TfidfVectorizer

# Divide el dataset en conjuntos de entrenamiento y prueba.
# Permite evaluar el modelo con datos no vistos.
from sklearn.model_selection import train_test_split

# Modelo de clasificación supervisada.
# Aprende la relación entre los vectores TF-IDF y el sentimiento.
from sklearn.linear_model import LogisticRegression

# Convierte etiquetas textuales (positivo, negativo, neutro) en valores numéricos.
# Necesario para entrenar modelos supervisados.
from sklearn.preprocessing import LabelEncoder

# Encadena múltiples pasos (vectorización + modelo).
# Garantiza consistencia entre entrenamiento y predicción.
# Facilita despliegue en APIs (FastAPI).
from sklearn.pipeline import Pipeline

# 'classification_report' es una función que genera un informe detallado de métricas de clasificación.
# Evalúa el modelo comparando: valores reales (y_test) y valores predichos (y_pred) y produce métricas por cada clase.

# 'confusion_matrix' es una herramienta para evaluar modelos de clasificación.

from sklearn.metrics import classification_report, confusion_matrix

## 2.1- Cargando el DataFrame

In [2]:
df = pd.read_csv("df_completo_fx.csv")

In [3]:
df['sentiment']=df['sentiment'].str.upper()

In [4]:
df.head(3)

,id,review_body,stars,language,lenght_review_body,review_body_clean,sentiment
0,0,"sí es totalmente impermeable, pero no se adapt...",3,es,144,sí es totalmente impermeable pero no se adapta...,NEUTRO
1,1,el precio esta bien del producto pero el embal...,3,es,174,el precio esta bien del producto pero el embal...,NEUTRO
2,2,De momento va muy bien y tiene un diseño muy e...,5,es,54,de momento va muy bien y tiene un diseño muy e...,POSITIVO


## 2.2- Visión general

### ¿Qué vamos a construir?

Un modelo supervisado de clasificación de texto que:

- Recibe un comentario en texto

- Transforma el texto en números con TF-IDF

- Predice el sentimiento con Regresión Logística

### Supuestos iniciales (estado del dataset)

Antes de comenzar, el DataFrame debería cumplir estas condiciones:

- Una columna con texto limpio → `'review_body_clean'`

- Una columna con la etiqueta → `'sentiment'`.

- Valores de `'sentiment'`: positivo, neutro, negativo.

- Dataset ya filtrado / muestreado.

Si no se cumple alguno, este modelo no debe entrenarse aún.

## 2.3- Definición de variables


Todo modelo supervisado necesita:

    X → variables de entrada (features)
    y → variable objetivo (label)

En NLP:

    X = texto
    y = sentimiento

In [5]:
X = df['review_body_clean']
y = df['sentiment']

## 2.4- Codificación de la variable objetivo

Transformación interna:

- negativo → 0
- neutro → 1
- positivo → 2

Esto no cambia el significado, solo facilita el entrenamiento.

In [6]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

## 2.5- División entrenamiento/prueba

`train_test_split` es una función de scikit-learn que divide un dataset en dos subconjuntos:
- Conjunto de entrenamiento (train)
- Conjunto de prueba (test)

El objetivo es evaluar la capacidad de generalización del modelo, es decir, qué tan bien predice datos que no ha visto durante el entrenamiento.

¿Qué hace cada parámetro?
- `test_size`=0.2 → 20% para evaluación, el 80% restante se usa para entrenamiento.
- `random_state`=42 → reproducibilidad.
- stratify=y → Obliga a que la proporción de clases en y se mantenga igual en:
    - y_train
    - y_test

In [7]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

## 2.6- Construcción del Pipeline TF-IDF + Regresión Logística

### ¿Qué es un Pipeline?

Pipeline es una abstracción de scikit-learn que permite encadenar varios pasos de procesamiento y modelado en un solo objeto.

En este caso, el pipeline define un flujo fijo:
1. Transformación del texto (TF-IDF)
2. Clasificación (Regresión Logística)

#### Por qué se usa un pipeline
- Evita data leakage
- Garantiza que el mismo procesamiento se aplique a entrenamiento, validación y predicción
- Simplifica el código y el despliegue

Es el estándar profesional en NLP con scikit-learn

### 2.6.2- TF-IDF

- `('tfidf', TfidfVectorizer(...))` → Este paso transforma texto en números, lo cual es obligatorio para que un modelo matemático pueda trabajar.

- `TfidfVectorizer` → TF-IDF significa Term Frequency – Inverse Document Frequency. Su función es:
  - Convertir cada comentario en un vector numérico
  - Asignar pesos a las palabras según:
    - su frecuencia en el texto
    - su rareza en el corpus completo
  
  Esto permite capturar importancia semántica, no solo conteos.
  
- `max_features=5000` → Limita el vocabulario a las 5000 palabras o n-gramas más relevantes. Reduce la dimensionalidad, el ruido y el consumo de memoria.

  Sin este límite, el vocabulario puede crecer a decenas o cientos de miles de términos, afectando el rendimiento y la estabilidad del modelo

- `ngram_range=(1, 2)` → Indica que el vectorizador utilizará:
  - Unigramas (1 palabra: bueno)
  - Bigramas (2 palabras consecutivas: muy bueno)

  Captura contexto básico, permite distinguir frases como “no funciona” vs “funciona”.
  
  Es un compromiso adecuado entre expresividad y complejidad.

 ### 2.6.3- Regresión Logística

Este paso es el modelo de Machine Learning propiamente dicho.

`LogisticRegression` → La Regresión Logística es un clasificador lineal, ampliamente utilizado en NLP porque:
- Funciona muy bien con TF-IDF
- Escala correctamente con texto de alta dimensión
- Produce probabilidades (predict_proba)
- Es interpretable

En nuestro caso, se usa en modo multiclase para: Negativo, neutro y positivo.

`max_iter=1000` →
- Define el número máximo de iteraciones del algoritmo de optimización.
- Se incrementa respecto al valor por defecto para asegurar convergencia y evitar advertencias en datasets grandes o complejos

Es una buena práctica estándar.

`n_jobs=-1` → Indica que el entrenamiento utilice todos los núcleos disponibles del CPU y reduce el tiempo de entrenamiento.

No cambia el resultado, solo el rendimiento.

In [8]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(
        max_features=5000,
        ngram_range=(1, 2),
    )),
    ('clf', LogisticRegression(
        max_iter=1000,
        n_jobs=-1
    ))
])


## 2.7- Entrenamiento del modelo

Cuando se llama `pipeline.fit(X_train, y_train)` ocurre lo siguiente en orden:
1. `TfidfVectorizer`:
    - Aprende el vocabulario
    - Calcula pesos TF-IDF
2. `LogisticRegression`:
    - Aprende los pesos que asocian palabras a sentimientos

In [9]:
pipeline.fit(X_train, y_train)

c:\Users\rauln\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:1184: FutureWarning: 'n_jobs' has no effect since 1.8 and will be removed in 1.10. You provided 'n_jobs=-1', please leave it unspecified.
  warnings.warn(msg, category=FutureWarning)


,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('tfidf', ...), ('clf', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"input input: {'filename', 'file', 'content'}, default='content'- If `'filename'`, the sequence passed as an argument to fit is expected to be a list of filenames that need reading to fetch the raw content to analyze.- If `'file'`, the sequence items must have a 'read' method (file-like object) that is called to fetch the bytes in memory.- If `'content'`, the input is expected to be a sequence of items that can be of type string or byte.",'content'
,"encoding encoding: str, default='utf-8'If bytes or files are given to analyze, this encoding is used todecode.",'utf-8'
,"decode_error decode_error: {'strict', 'ignore', 'replace'}, default='strict'Instruction on what to do if a byte sequence is given to analyze thatcontains characters not of the given `encoding`. By default, it is'strict', meaning that a UnicodeDecodeError will be raised. Othervalues are 'ignore' and 'replace'.",'strict'
,"strip_accents strip_accents: {'ascii', 'unicode'} or callable, default=NoneRemove accents and perform other character normalizationduring the preprocessing step.'ascii' is a fast method that only works on characters that havea direct ASCII mapping.'unicode' is a slightly slower method that works on any characters.None (default) means no character normalization is performed.Both 'ascii' and 'unicode' use NFKD normalization from:func:`unicodedata.normalize`.",None
,"lowercase lowercase: bool, default=TrueConvert all characters to lowercase before tokenizing.",True
,"preprocessor preprocessor: callable, default=NoneOverride the preprocessing (string transformation) stage whilepreserving the tokenizing and n-grams generation steps.Only applies if ``analyzer`` is not callable.",None
,"tokenizer tokenizer: callable, default=NoneOverride the string tokenization step while preserving thepreprocessing and n-grams generation steps.Only applies if ``analyzer == 'word'``.",None


---
---

## 2.8- Salida de predicción del modelo con texto de prueba

Esta función recibe un `texto`(string) individual, utiliza un pipeline entrenado (**TF-IDF + Regresión Logística**) y devuelve:
- Sentimiento predicho
- Score asociado a esa predicción

`pipeline` → Es el modelo ya entrenado, contiene:
- `TfidfVectorizer`
- `LogisticRegression`

Debe haber pasado previamente por:
- `pipeline.fit(X_train, y_train)`

`predict_proba` → Convierte el texto en un vector **TF-IDF** y calcula la probabilidad de cada clase

`pipeline.classes_` → Devuelve el orden exacto de las clases aprendidas por el modelo

`idx = probs.argmax()` → Identifica la posición de la probabilidad más alta y devuelve el índice de la clase más probable



In [10]:
def predecir_sentimiento_score(texto, pipeline):
    probs = pipeline.predict_proba([texto])[0]
    classes = pipeline.classes_

    idx = probs.argmax()

    return {
        "prevision": classes[idx],
        "probabilidad": float(round(probs[idx],2))
    }

In [11]:
predecir_sentimiento_score(
    "el producto funciona bien pero el envío fue muy lento",
    pipeline
)

{'prevision': 'NEUTRO', 'probabilidad': 0.6}

### 2.8- Interpretación correcta del score

Score alto (≈ 0.8 – 1.0)
- El modelo está muy seguro

Score medio (≈ 0.5 – 0.7)
- sentimiento razonable, pero con ambigüedad

Score bajo (< 0.5)
- texto ambiguo o cercano a otra clase

---
---

## 2.9- Evaluación del modelo

`classification_report` es una función que genera un informe detallado de métricas de clasificación.

Evalúa el modelo comparando:
- valores reales (y_test)
- valores predichos (y_pred)

Y produce métricas por cada clase.

Para cada clase (Negativo, Neutro, Positivo), calcula:
- **Precision**: De todas las predicciones positivas para una clase cuántas fueron correctas; evalúa calidad de la predicción.
- **Recall**: De todos los casos reales de una clase, cuántos detectó el modelo; evalúa capacidad de detección.
- **F1-score**: Media armónica entre precision y recall. Es la métrica más usada en NLP porque penaliza falsos positivos y falsos negativos, además es robusta ante desbalance de clases
- **Support**: Es número real de muestras por clase en el conjunto de prueba. Permite interpretar la confiabilidad de las métricas.

In [12]:
y_pred = pipeline.predict(X_test)

In [13]:
print(classification_report(y_test, y_pred))
confusion_matrix(y_test, y_pred)

              precision    recall  f1-score   support

    NEGATIVO       0.75      0.85      0.80     15840
      NEUTRO       0.47      0.27      0.35      7903
    POSITIVO       0.78      0.85      0.81     15665

    accuracy                           0.73     39408
   macro avg       0.67      0.66      0.65     39408
weighted avg       0.71      0.73      0.71     39408



array([[13467,  1287,  1086],
       [ 3172,  2163,  2568],
       [ 1237,  1172, 13256]])

### 2.9.1- Exportar el reporte de métricas  a formato .csv

Compara:
- `y_tes`t → etiquetas reales
- `y_pred` → etiquetas predichas por el modelo

Calcula métricas de clasificación:Precision, recall, f1-score y support

`output_dict=True` → Cambia el formato de salida a diccionario estructurado

In [14]:
report_dict = classification_report(
    y_test,
    y_pred,
    output_dict=True
)

df_report = pd.DataFrame(report_dict).transpose()
df_report = df_report.round(3)

In [15]:
df_report.to_csv("metrics.csv", index=True)

### 2.9.2- Exportando la matriz de confusión

La matriz de confusión es una tabla que muestra:
- Qué tan bien predice el modelo
- Dónde se equivoca
- Qué clases confunde entre sí

Compara directamente:
- Valores reales (y_test)
- Valores predichos (y_pred)

| Real \ Predicho | Negativo  | Neutro  | Positivo  |
| --------------- | --------- | ------- | --------- |
| **Negativo**    | TPₙ       | FNₙ→neu | FNₙ→pos   |
| **Neutro**      | FNₙeu→neg | TPₙeu   | FNₙeu→pos |
| **Positivo**    | FNₚ→neg   | FNₚ→neu | TPₚ       |

Donde:

- Diagonal principal → predicciones correctas
- Fuera de la diagonal → errores del modelo

Relación con precision, recall y F1

Todas las métricas del classification_report salen de la matriz de confusión:
- Precision → columnas
- Recall → filas
- F1-score → combinación de ambas

La matriz es la fuente primaria de evaluación.

In [16]:
cm = confusion_matrix(y_test, y_pred)

In [17]:
df_cm = pd.DataFrame(
    cm,
    index=pipeline.classes_,
    columns=pipeline.classes_
)

In [18]:
df_cm.to_csv("confusion_matrix.csv")

# 3- Serialización del modelo

In [19]:
import joblib

# Guardar el vectorizador TF-IDF
joblib.dump(pipeline, "sentiment_pipeline.joblib")

['sentiment_pipeline.joblib']

In [20]:
pipeline.predict(X_train[:5])

array(['NEGATIVO', 'NEUTRO', 'NEGATIVO', 'POSITIVO', 'NEGATIVO'],
      dtype=object)

In [21]:
pipeline_loaded = joblib.load("sentiment_pipeline.joblib")

pipeline_loaded.predict(["El servicio fue excelente y muy rápido"])

array(['POSITIVO'], dtype=object)